In [1]:
#!pip install -q git+https://github.com/huggingface/transformers.git

You should consider upgrading via the '/home/ali/.virtualenvs/depth/bin/python -m pip install --upgrade pip' command.


In [13]:
from transformers import GLPNFeatureExtractor, GLPNForDepthEstimation
import torch
feature_extractor = GLPNFeatureExtractor.from_pretrained("vinvino02/glpn-kitti")
model = GLPNForDepthEstimation.from_pretrained("vinvino02/glpn-kitti")

In [21]:
import os, sys, errno
import time
import glob
import PIL.Image as pil
from timer import Timer
import cv2
import numpy as np

In [15]:
paths= sorted(glob.glob(os.path.join("./lp-left", '*.{}'.format("png"))))
paths_out="results/glpn-kitti"
os.makedirs(paths_out, exist_ok=True)

In [23]:
import matplotlib as mpl
import matplotlib.cm as cm


In [27]:
star=time.time()
cont=0
timer = Timer()
for idx, image_path in enumerate(paths):# Use the detector to do inference
    timer.tic()
    image=pil.open(image_path).convert('RGB')
    pixel_values = feature_extractor(image, return_tensors="pt").pixel_values
    with torch.no_grad():
        outputs = model(pixel_values)
        predicted_depth = outputs.predicted_depth
    
    prediction = torch.nn.functional.interpolate(
                    predicted_depth.unsqueeze(1),
                    size=pixel_values.shape[-2:],
                    mode="bicubic",
                    align_corners=False,
             )
    
    disp_resized_np = prediction.squeeze().cpu().numpy()
    timer.toc()
    vmax = np.percentile(disp_resized_np, 95)
    normalizer = mpl.colors.Normalize(vmin=disp_resized_np.min(), vmax=vmax)
    mapper = cm.ScalarMappable(norm=normalizer, cmap='magma')
    colormapped_im = (mapper.to_rgba(disp_resized_np)[:, :, :3] * 255).astype(np.uint8)
    
    text_scale = 2
    text_thickness = 2
    line_thickness = 1
    time_avg=timer.average_time
    if time_avg==0:
                time_avg=1
        
    fps=max(1e-10, 1. / time_avg)
    cv2.putText(colormapped_im, 'fps: %.2f' % fps,
                (0, int(15 * text_scale)), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), thickness=2)
    im = pil.fromarray(colormapped_im)
    
    output_name = os.path.splitext(os.path.basename(image_path))[0]
    name_deth_pro = os.path.join(paths_out,"{}.png".format(output_name))
    
    im.save(name_deth_pro)
    fin=time.time()
   
    #show_result_pyplot(model, image_path, result, score_thr=0.3)
    #show_result_pyplot(model, image_path, result, score_thr=0)
    print((idx+1)/(fin-star)," foto ",idx," time ",fin-star,"fps", 1. / max(1e-5, timer.average_time))

0.7842852152944353  foto  0  time  1.2750463485717773 fps 0.8091146700916834
0.7176909547977303  foto  1  time  2.786714792251587 fps 0.7375310414497044
0.6870267069235938  foto  2  time  4.366642475128174 fps 0.7048285377621125
0.6789839742904856  foto  3  time  5.891155242919922 fps 0.6962198945935737
0.6738588575909438  foto  4  time  7.419951438903809 fps 0.690780413298519
0.6621327210987807  foto  5  time  9.061627388000488 fps 0.6784304149442719
0.6494967837661199  foto  6  time  10.77757453918457 fps 0.6653934500096568
0.6242252188392444  foto  7  time  12.815887212753296 fps 0.6388254627136558
0.6227828661895731  foto  8  time  14.45126461982727 fps 0.6373090939995871
0.621330412867392  foto  9  time  16.094496250152588 fps 0.6358315660245335
0.6229226816909758  foto  10  time  17.658692359924316 fps 0.6378589004474021
0.619459689577535  foto  11  time  19.371720552444458 fps 0.6343416430543988
0.6199049393367809  foto  12  time  20.970957279205322 fps 0.6348753869950566
0.6200

0.5944428495797575  foto  107  time  181.68273043632507 fps 0.6102152532426772
0.5947863037521786  foto  108  time  183.25909543037415 fps 0.6105726102539121
0.5951004757819722  foto  109  time  184.8427357673645 fps 0.6109019199612936
0.5951422390516145  foto  110  time  186.51003527641296 fps 0.6109671324559159
0.5956123804796684  foto  111  time  188.0417594909668 fps 0.611459336063775
0.5960258289924373  foto  112  time  189.58909916877747 fps 0.6118841591606708
0.5962085594206852  foto  113  time  191.20825791358948 fps 0.6120604911040429
0.5963098993572288  foto  114  time  192.8527433872223 fps 0.6121554226659509
0.5964737917632967  foto  115  time  194.47627305984497 fps 0.6123166763270652
0.5965853426165696  foto  116  time  196.11611557006836 fps 0.6124196437963723
0.5967070495552198  foto  117  time  197.75197911262512 fps 0.6125349279176285
0.5968080379236901  foto  118  time  199.39409732818604 fps 0.6126430017506573
0.5969790473655766  foto  119  time  201.01207995414734 

0.6069883581500666  foto  212  time  350.9128258228302 fps 0.6229911428892134
0.607259707038853  foto  213  time  352.40276527404785 fps 0.6232861216837916
0.6074274042565716  foto  214  time  353.9517619609833 fps 0.6234711062897799
0.6074421240238036  foto  215  time  355.58943223953247 fps 0.6234902957859996
0.607379470098317  foto  216  time  357.27253007888794 fps 0.6234371511628475
0.6075446565734515  foto  217  time  358.8213601112366 fps 0.6236099382985032
0.6077577517728971  foto  218  time  360.34094071388245 fps 0.6238367590084039
0.6079385472252892  foto  219  time  361.8786816596985 fps 0.6240318707067435
0.6080140635190499  foto  220  time  363.4784345626831 fps 0.6241189909729923
0.6079475183343741  foto  221  time  365.16309928894043 fps 0.6240539338340545
0.607846153038011  foto  222  time  366.8691475391388 fps 0.6239550058835955
0.6080362809276647  foto  223  time  368.3990693092346 fps 0.6241573525501283
0.6081232338704466  foto  224  time  369.9907970428467 fps 0.6

0.613531931788981  foto  317  time  518.3104310035706 fps 0.6301425038698178
0.6136408279010023  foto  318  time  519.8480699062347 fps 0.6302590216135701
0.6137628324023624  foto  319  time  521.3740277290344 fps 0.630384226798616
0.6138799237837217  foto  320  time  522.9035639762878 fps 0.6305097260594338
0.613982413918445  foto  321  time  524.444988489151 fps 0.6306211862050092
0.6140839169985374  foto  322  time  525.986743927002 fps 0.6307294619512751
0.6141736067851248  foto  323  time  527.5381364822388 fps 0.6308244841674693
0.6142447672613893  foto  324  time  529.1050364971161 fps 0.6308997865733775
0.6142788075299713  foto  325  time  530.7036414146423 fps 0.6309327258486982
0.6142464015281042  foto  326  time  532.3596510887146 fps 0.6309000981551198
0.6142371650274929  foto  327  time  533.9956920146942 fps 0.6308912078821222
0.6143220191084754  foto  328  time  535.5497438907623 fps 0.630980711698036
0.6144026547387434  foto  329  time  537.107054233551 fps 0.6310696035

0.6183675839768705  foto  423  time  685.6763048171997 fps 0.6354149830742076
0.6183583892594886  foto  424  time  687.3036856651306 fps 0.6354072748123724
0.6183677670602236  foto  425  time  688.91042304039 fps 0.6354200765776759
0.6183608509433539  foto  426  time  690.535306930542 fps 0.6354223187029903
0.6183628723466427  foto  427  time  692.1502230167389 fps 0.6354276744938512
0.6183692530809944  foto  428  time  693.7602376937866 fps 0.635435263772613
0.6184969054802001  foto  429  time  695.2338745594025 fps 0.6355721139700968
0.6185617331203689  foto  430  time  696.777664899826 fps 0.6356429627332428
0.6186453006611541  foto  431  time  698.2999782562256 fps 0.6357328023829545
0.6187304759608557  foto  432  time  699.8200619220734 fps 0.6358260834703908
0.6187916339813322  foto  433  time  701.3669483661652 fps 0.6358921368193979
0.6188600188731248  foto  434  time  702.9053206443787 fps 0.6359667524120305
0.6188278594672587  foto  435  time  704.5578076839447 fps 0.63593436

0.6215360179647367  foto  529  time  852.7261247634888 fps 0.6390566918299001
0.6215816227485965  foto  530  time  854.2723603248596 fps 0.639108328809688
0.621620558136691  foto  531  time  855.8275511264801 fps 0.6391529878504867
0.6216600921578251  foto  532  time  857.381721496582 fps 0.639197736817954
0.6216451398664635  foto  533  time  859.0109786987305 fps 0.6391829275896259
0.6216219220545999  foto  534  time  860.6517579555511 fps 0.63915986507341
0.6216869920828829  foto  535  time  862.1702027320862 fps 0.639232464195869
0.621688068018271  foto  536  time  863.7772343158722 fps 0.6392371780486407
0.6217904098061592  foto  537  time  865.2433223724365 fps 0.639350519639425
0.6218247551734682  foto  538  time  866.8037023544312 fps 0.6393899252521135
0.6218120545098853  foto  539  time  868.4296100139618 fps 0.6393807550889751
0.6217630037677769  foto  540  time  870.1064500808716 fps 0.6393307204513676
0.6218192399565511  foto  541  time  871.6359436511993 fps 0.639392418911

0.6237664933000817  foto  635  time  1019.6123178005219 fps 0.6415000491287015
0.6237742399688178  foto  636  time  1021.202799320221 fps 0.6415061670098714
0.6237516599085492  foto  637  time  1022.842969417572 fps 0.641480215964091
0.6237857011556774  foto  638  time  1024.3902654647827 fps 0.641514861122641
0.6237674664754365  foto  639  time  1026.0233731269836 fps 0.6414945048851026
0.6237590609498498  foto  640  time  1027.640382528305 fps 0.6414854721879923
0.6237150211494303  foto  641  time  1029.3162393569946 fps 0.6414358610397495
0.6236930986870538  foto  642  time  1030.9557719230652 fps 0.6414105478553965
0.6237529666334267  foto  643  time  1032.4600193500519 fps 0.6414762143179227
0.623716970558897  foto  644  time  1034.122896194458 fps 0.6414348559239705
0.6237513373149933  foto  645  time  1035.6691222190857 fps 0.6414702379859738
0.6237311235868903  foto  646  time  1037.3059408664703 fps 0.6414461600329763
0.6237931503817705  foto  647  time  1038.8058919906616 fps

0.6253359991673366  foto  740  time  1184.9629654884338 fps 0.6430791260390347
0.6253478747875221  foto  741  time  1186.5395724773407 fps 0.6430885728299663
0.6253674894883634  foto  742  time  1188.101416349411 fps 0.6431093284184196
0.6253991530563088  foto  743  time  1189.6402423381805 fps 0.6431444444496633
0.6254182510280959  foto  744  time  1191.2028450965881 fps 0.6431627451113224
0.6254360181648297  foto  745  time  1192.7678904533386 fps 0.6431791215456722
0.6254116740398197  foto  746  time  1194.4132657051086 fps 0.6431536616154582
0.6253798510228591  foto  747  time  1196.0730726718903 fps 0.6431201895370644
0.6253881140347574  foto  748  time  1197.6562764644623 fps 0.643126947576168
0.6253857464958509  foto  749  time  1199.2598235607147 fps 0.6431233246579003
0.6254394939441454  foto  750  time  1200.7556402683258 fps 0.6431802262334397
0.6254429060949349  foto  751  time  1202.347956418991 fps 0.6431842242167005
0.6254520939525007  foto  752  time  1203.929137468338 